# Ollama

In [19]:
!pip install -U langchain-nomic langchain_community tiktoken langchainhub chromadb langchain langgraph tavily-python gpt4all webdriver_manager selenium groq langchain_groq cassio amazon-textract-caller

In [20]:
local_llm = 'llama3'

In [21]:
### Index
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings
###############################################################################################################

from langchain_community.document_loaders import PyPDFLoader

pdfs_links = [
    "Politica_Derechos_Humanos.pdf",
    "Politica_Prevencion_Acoso_Laboral.pdf",
    "Politica_Proteccion_Datos_Personales.pdf",
    "Politica_SGC.pdf",
    "Politica_SGSST.pdf"
]


pdfs = [PyPDFLoader(url).load() for url in pdfs_links]

pdfs_docs  = [item for sublist in pdfs for item in sublist]

###########################################################################################################
# from langchain_community.document_loaders import csv_loader

# # Convertir las cadenas de texto en objetos Document
# docs_list = csv_loader.CSVLoader("messages.csv", source_column="Message")
# csv_docs = docs_list.load()
###########################################################################################################
# Descomenta estas líneas si deseas cargar documentos de las URLs
urls = [
    "https://www.finanzauto.com.co/portal/pqrinter",
    "https://www.finanzauto.com.co/portal/ubicacion",
    "https://www.finanzauto.com.co/portal/terminos",
]

docs = [WebBaseLoader(url).load() for url in urls]
web_docs  = [item for sublist in docs for item in sublist]


# Combina todos los documentos en una lista
all_docs = pdfs_docs + web_docs # + csv_docs

# Divide los documentos en fragmentos de texto
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=150, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(all_docs)


# Ahora puedes pasar docs_list a Chroma.from_documents()
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=GPT4AllEmbeddings(),
)
retriever = vectorstore.as_retriever()


In [22]:
### Retrieval Grader 

from langchain_groq import ChatGroq
#############################################################################################################

from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
#########################################################################################################################

llm=ChatGroq(groq_api_key="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",
         model_name="llama3-70b-8192")

from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""<|begin_of_text|><|start_header_id|>system<|end_header_id|> Eres un calificador evaluando la relevancia de un documento recuperado para una pregunta del usuario. Si el documento contiene palabras clave relacionadas con la pregunta del usuario, califícalo como relevante. No es necesario que sea una prueba estricta. \n
Proporciona una puntuación binaria 'yes' o 'no' para indicar si el documento es relevante para la pregunta. \n
Proporciona la puntuación binaria como un JSON con una sola clave 'score' y sin preámbulo ni explicación. \n
Aquí tienes el documento recuperado: \n\n {document} \n\n
Aquí está la pregunta del usuario: {question} \n <|eot_id|><|start_header_id|>assistant<|end_header_id|>
#     """)

#########################################################################################################################
## LLM
# llm = ChatOllama(model=local_llm, format="json", temperature=0)

# prompt = PromptTemplate(
#     template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing relevance 
#     of a retrieved document to a user question. If the document contains keywords related to the user question, 
#     grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
#     Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
#     Provide the binary score as a JSON with a single key 'score' and no premable or explaination.
#      <|eot_id|><|start_header_id|>user<|end_header_id|>
#     Here is the retrieved document: \n\n {document} \n\n
#     Here is the user question: {question} \n <|eot_id|><|start_header_id|>assistant<|end_header_id|>
#     """,
#     input_variables=["question", "document"],
# )

retrieval_grader = prompt | llm | JsonOutputParser()
question = "Cual es el horario de finanzauto?"
docs = retriever.invoke(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

{'score': 'yes'}


In [23]:
### Generate

from langchain.prompts import PromptTemplate
from langchain import hub
from langchain_core.output_parsers import StrOutputParser

#####################################################################################################################


llm=ChatGroq(groq_api_key=os.environ['GROQ_API_KEY'],
         model_name="llama3-70b-8192")

from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""<|begin_of_text|><|start_header_id|>system<|end_header_id|>Eres un asistente para tareas de preguntas y respuestas. 
    Usa los siguientes fragmentos de contexto recuperado para responder la pregunta. Si no sabes la respuesta, simplemente di que no lo sabes.
    Utiliza un máximo de tres oraciones y mantén la respuesta concisa.  <|eot_id|><|start_header_id|>user<|end_header_id|>
    Pregunta: {question} Contexto: {context} 
    Respuesta: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""")


#####################################################################################################################
# # Prompt
# prompt = PromptTemplate(
#     template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant for question-answering tasks. 
#     Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. 
#     Use three sentences maximum and keep the answer concise <|eot_id|><|start_header_id|>user<|end_header_id|>
#     Question: {question} 
#     Context: {context} 
#     Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
#     input_variables=["question", "document"],
# )

# llm = ChatOllama(model=local_llm, temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
question = "Que es finanzauto?"
docs = retriever.invoke(question)
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

Finanzauto es una plataforma que ofrece una amplia gama de soluciones financieras diseñadas para cubrir necesidades económicas de manera rápida y sencilla.


In [24]:
### Hallucination Grader 

#####################################################################################################################


llm=ChatGroq(groq_api_key=os.environ['GROQ_API_KEY'],
         model_name="llama3-70b-8192")

from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(""" <|begin_of_text|><|start_header_id|>system<|end_header_id|> Eres un evaluador que determina si una respuesta
    está fundamentada o respaldada por un conjunto de hechos. Proporciona una puntuación binaria 'yes' o 'no' para indicar si la respuesta está fundamentada
    o respaldada por un conjunto de hechos. Proporciona la puntuación binaria como un JSON con una sola clave 'score' y sin preámbulo o explicación. <|eot_id|><|start_header_id|>user<|end_header_id|>
    Aquí están los hechos: 
    \n ------- \n
    {documents} 
    \n ------- \n
    Aquí está la respuesta: {generation}<|eot_id|><|start_header_id|>assistant<|end_header_id|>""")


#####################################################################################################################


# # LLM
# llm = ChatOllama(model=local_llm, format="json", temperature=0)

# # Prompt
# prompt = PromptTemplate(
#     template=""" <|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether 
#     an answer is grounded in / supported by a set of facts. Give a binary score 'yes' or 'no' score to indicate 
#     whether the answer is grounded in / supported by a set of facts. Provide the binary score as a JSON with a 
#     single key 'score' and no preamble or explanation. <|eot_id|><|start_header_id|>user<|end_header_id|>
#     Here are the facts:
#     \n ------- \n
#     {documents} 
#     \n ------- \n
#     Here is the answer: {generation}  <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
#     input_variables=["generation", "documents"],
# )

hallucination_grader = prompt | llm | JsonOutputParser()
hallucination_grader.invoke({"documents": docs, "generation": generation})

{'score': 'yes'}

In [25]:
### Answer Grader 

#####################################################################################################################


llm=ChatGroq(groq_api_key=os.environ['GROQ_API_KEY'],
         model_name="llama3-70b-8192")

from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""<|begin_of_text|><|start_header_id|>system<|end_header_id|> 
Eres un evaluador que determina si una respuesta es útil para resolver una pregunta. Proporciona una puntuación binaria 'yes' o 'no' 
para indicar si la respuesta es útil para resolver una pregunta. Proporciona la puntuación binaria como un JSON con una sola clave
'score' y sin preámbulo o explicación. <|eot_id|><|start_header_id|>user<|end_header_id|> 
Aquí está la respuesta: 
\n ------- \n 
{generation}
\n ------- \n 
Aquí está la pregunta: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""")


#####################################################################################################################


# # LLM
# llm = ChatOllama(model=local_llm, format="json", temperature=0)

# # Prompt
# prompt = PromptTemplate(
#     template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether an 
#     answer is useful to resolve a question. Give a binary score 'yes' or 'no' to indicate whether the answer is 
#     useful to resolve a question. Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.
#      <|eot_id|><|start_header_id|>user<|end_header_id|> Here is the answer:
#     \n ------- \n
#     {generation} 
#     \n ------- \n
#     Here is the question: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
#     input_variables=["generation", "question"],
# )

answer_grader = prompt | llm | JsonOutputParser()
answer_grader.invoke({"question": question,"generation": generation})

{'score': 'yes'}

In [26]:
### Router

from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser


#####################################################################################################################


llm=ChatGroq(groq_api_key=os.environ['GROQ_API_KEY'],
         model_name="llama3-70b-8192")

from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""<|begin_of_text|><|start_header_id|>system<|end_header_id|> Eres un experto en encaminar una pregunta
de usuario a un vectorstore o web search. Utiliza el vectorstore para preguntas sobre Finanzauto o una pregunta de servicio al cliente en general. 
No es necesario ser estricto con las palabras clave en la pregunta relacionadas con estos temas. De lo contrario, utiliza la web search.
Proporciona una elección binaria 'web_search' o 'vectorstore' basada en la pregunta. Devuelve un JSON con una sola clave 'datasource'
y sin preámbulo ni explicación. Pregunta a enrutar: {question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>""")


#####################################################################################################################


# # LLM
# llm = ChatOllama(model=local_llm, format="json", temperature=0)

# prompt = PromptTemplate(
#     template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an expert at routing a 
#     user question to a vectorstore or web search. Use the vectorstore for questions on LLM  agents, 
#     prompt engineering, and adversarial attacks. You do not need to be stringent with the keywords 
#     in the question related to these topics. Otherwise, use web-search. Give a binary choice 'web_search' 
#     or 'vectorstore' based on the question. Return the a JSON with a single key 'datasource' and 
#     no premable or explaination. Question to route: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
#     input_variables=["question"],
# )

question_router = prompt | llm | JsonOutputParser()
question = "Donde puedo ver los horarios de Finanzauto"
docs = retriever.get_relevant_documents(question)
doc_txt = docs[1].page_content
print(question_router.invoke({"question": question}))

{'datasource': 'vectorstore'}


In [27]:
### Search

from langchain_community.tools.tavily_search import TavilySearchResults
web_search_tool = TavilySearchResults(k=3)


In [28]:
from typing_extensions import TypedDict
from typing import List

### State

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        web_search: whether to add search
        documents: list of documents 
    """
    question : str
    generation : str
    web_search : str
    documents : List[str]

from langchain.schema import Document

### Nodes

def retrieve(state):
    """
    Retrieve documents from vectorstore

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question}

def generate(state):
    """
    Generate answer using RAG on retrieved documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]
    
    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}

def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question
    If any document is not relevant, we will set a flag to run web search

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Filtered out irrelevant documents and updated web_search state
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]
    
    # Score each doc
    filtered_docs = []
    web_search = "no"
    for d in documents:
        score = retrieval_grader.invoke({"question": question, "document": d.page_content})
        grade = score['score']
        # Document relevant
        if grade.lower() == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        # Document not relevant
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            # We do not include the document in filtered_docs
            # We set a flag to indicate that we want to run web search
            web_search = "yes"
            continue
        
        if filtered_docs is not None:
            web_search = "no"


    return {"documents": filtered_docs, "question": question, "web_search": web_search}
    
def web_search(state):
    """
    Web search based based on the question

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Appended web results to documents
    """

    print("---WEB SEARCH---")
    question = state["question"]
    documents = state["documents"]

    # Web search
    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)
    if documents is not None:
        documents.append(web_results)
    else:
        documents = [web_results]
    return {"documents": documents, "question": question}

### Conditional edge

def route_question(state):
    """
    Route question to web search or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("---ROUTE QUESTION---")
    question = state["question"]
    print(question)
    source = question_router.invoke({"question": question})  
    print(source)
    print(source['datasource'])
    if source['datasource'] == 'web_search':
        print("---ROUTE QUESTION TO WEB SEARCH---")
        return "websearch"
    elif source['datasource'] == 'vectorstore':
        print("---ROUTE QUESTION TO RAG---")
        return "vectorstore"

def decide_to_generate(state):
    """
    Determines whether to generate an answer, or add web search

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    question = state["question"]
    web_search = state["web_search"]
    filtered_documents = state["documents"]

    if web_search == "yes":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print("---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---")
        return "websearch"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"

### Conditional edge

def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke({"documents": documents, "generation": generation})
    grade = score['score']

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question,"generation": generation})
        grade = score['score']
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

from langgraph.graph import END, StateGraph
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("websearch", web_search) # web search
workflow.add_node("retrieve", retrieve) # retrieve
workflow.add_node("grade_documents", grade_documents) # grade documents
workflow.add_node("generate", generate) # generatae

In [29]:
# Build graph
workflow.set_conditional_entry_point(
    route_question,
    {
        "websearch": "websearch",
        "vectorstore": "retrieve",
    },
)

workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "websearch": "websearch",
        "generate": "generate",
    },
)
workflow.add_edge("websearch", "generate")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "websearch",
    },
)

In [30]:
# Compile
app = workflow.compile()

# Test
from pprint import pprint
inputs = {"question": "dime el nombre del archivo donde encientro el trataiento de datos personales y cual es el link donde puedo entontrar el archivo?"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
pprint(value["generation"])

---ROUTE QUESTION---
dime el nombre del archivo donde encientro el trataiento de datos personales y cual es el link donde puedo entontrar el archivo?
{'datasource': 'vectorstore'}
vectorstore
---ROUTE QUESTION TO RAG---
---RETRIEVE---
'Finished running: retrieve:'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
'Finished running: grade_documents:'
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
'Finished running: generate:'
('El nombre del archivo es "Politica_Proteccion_Datos_Personales.pdf". No '
 'puedo proporcionar un enlace para descargar el archivo, ya que no tengo '
 'acceso a la ubicación física del archivo.')
